In [1]:
import torch
import numpy as np
import datasets
import pickle
import pathlib
import os
from importlib import reload
import sys
import pandas as pd
from IPython.display import display, HTML
import json
import sklearn
from sklearn.metrics import accuracy_score
import scipy
import matplotlib.pyplot as plt
import matplotlib
import logging

from tqdm.auto import tqdm
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers.trainer_pt_utils import LengthGroupedSampler
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding
)

# For Imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# print("sys.path:", sys.path)
plt.style.use("ggplot")

2023-01-03 19:33:39.799263: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 19:33:39.964226: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-03 19:33:40.662901: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/modaresi/.conda/envs/globenc-venv/lib/
2023-01-03 19:33:40.663002: W tensorflow/

In [90]:
MODELS_DIR = "/home/modaresi/projects/globenc_analysis/outputs/models"
OUTPUT_DIR = "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3"
IG_OUTPUT_DIR = "/home/modaresi/projects/globenc_analysis/outputs/integrated_saliencies"

DIR_MODEL_DATASET_SET = [
    (OUTPUT_DIR, f"{MODELS_DIR}/output_hatexplain_bert-base-uncased_0001_SEED0042/checkpoint-2405", "hatexplain", "validation"),
#     (f"{MODELS_DIR}/output_cola_bert-base-uncased_0001_SEED0042/checkpoint-1340", "cola", "validation"),
#     (OUTPUT_DIR, f"{MODELS_DIR}/output_sst2_bert-base-uncased_0001_SEED0042/checkpoint-10525", "sst2", "validation"),
#     (f"{MODELS_DIR}/output_mrpc_bert-base-uncased_0001_SEED0042/checkpoint-575", "mrpc", "validation"),
#     (f"{MODELS_DIR}/output_qnli_bert-base-uncased_0001_SEED0042/checkpoint-16370", "qnli", "validation"),
#     (f"{MODELS_DIR}/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-61360", "mnli", "validation_matched"),
#     (f"{MODELS_DIR}/output_sst2_bert-large-uncased_0001_SEED0042/checkpoint-10525", "sst2", "validation"),
### IG ###
#     (IG_OUTPUT_DIR, f"{MODELS_DIR}/output_sst2_bert-base-uncased_0001_SEED0042/checkpoint-10525", "sst2", "validation"),
]

GLOBENC_CONFIG_NAMES = [
#     "Decomposition LinearApproximation", 
    "Decomposition ZeroOriginApproximation", 
#     "GlobEnc",
#     "GlobEnc FFN LinearApproximation",
#     "GlobEnc FFN ZeroOriginApproximation",
### IG ###
#     "IG_ALL_PAD_SUM_label_based",
#     "IG_ALL_PAD_NORM_label_based",
#     "IG_ALL_ZERO_SUM_label_based",
#     "IG_ALL_ZERO_NORM_label_based",
#     "IG_ALL_PAD_SUM_prediction_based",
#     "IG_ALL_PAD_NORM_prediction_based",
#     "IG_ALL_ZERO_SUM_prediction_based",
#     "IG_ALL_ZERO_NORM_prediction_based",
]

In [91]:
def print_importance(importance, tokenized_text, discrete=False, prefix="", no_cls_sep=False):
    """
    importance: (sent_len)
    """
    if no_cls_sep:
        importance = importance[1:-1]
        tokenized_text = tokenized_text[1:-1]
    importance = importance / importance.max()  # Normalize
    if discrete:
        importance = np.argsort(np.argsort(importance)) / len(importance)
    
    html = "<pre>"+prefix
    for i in range(len(tokenized_text)):
        rgba = matplotlib.cm.get_cmap('Wistia')(importance[i])   # Wistia
        color = f"background-color: rgba({rgba[0]*255}, {rgba[1]*255}, {rgba[2]*255}, {rgba[3]}); "
        html += (f"<span style='"
                 f"{color}"
                 f"font-weight: {int(800)};"
                 "'>")
        html += tokenized_text[i]
        html += "</span> "
    display(HTML(html))

In [92]:
# Read Globencs
# globencs = dict()
NO_CLS_SEP = False
for dir_model_dataset_set in tqdm(DIR_MODEL_DATASET_SET, desc="Models_Dataset_Sets"):
    dir, model_checkpoint, task_name, set_of_data = dir_model_dataset_set
    for cfg_name in tqdm(GLOBENC_CONFIG_NAMES, desc="Configs"):
        file_name = f"[{task_name}]_[{set_of_data}]_[{'-'.join(model_checkpoint.split('/')[-2:])}]_[{cfg_name}]"
        
        try:
            df = pd.read_pickle(f"{dir}/{file_name}.pkl")
        except Exception as e:
            logging.warn(e)
            continue
        
        print(file_name)
        idx = 50
        for col in ["importance_last_layer_aggregated", "importance_last_layer_classifier"]:
            if col in df and df[col][idx] is not None:
                if "aggregated" in col:
                    sentence_importance = df[col].iloc[idx][0, :]
                if "classifier" in col:
                    for label in range(df[col].iloc[idx].shape[-1]):
                        sentence_importance = df[col].iloc[idx][:, label]
                        print_importance(
                            sentence_importance,
                            df["tokens"].iloc[idx], 
                            prefix=f"{col.split('_')[-1]}L{label}|L={df['label'].iloc[idx]}:".ljust(20),
                            no_cls_sep=NO_CLS_SEP,
                            discrete=False
                        )
                    break
                    sentence_importance = df[col].iloc[idx][:, df["label"].iloc[idx]]
                if "pooler" in col:
                    sentence_importance = df[col].iloc[idx]
                print_importance(
                    sentence_importance,
                    df["tokens"].iloc[idx], 
                    prefix=f"{col.split('_')[-1]}:".ljust(20),
                    no_cls_sep=NO_CLS_SEP,
                    discrete=False
                )
df.head(1)

Models_Dataset_Sets:   0%|          | 0/1 [00:00<?, ?it/s]

Configs:   0%|          | 0/1 [00:00<?, ?it/s]

[hatexplain]_[validation]_[output_hatexplain_bert-base-uncased_0001_SEED0042-checkpoint-2405]_[Decomposition ZeroOriginApproximation]


,tokens,importance_last_layer_aggregated,importance_last_layer_pooler,importance_last_layer_classifier,importance_all_layers_aggregated,cls,logits,label
0,"[[CLS], me, getting, books, from, the, library...","[[7.057031, 8.445731, 8.206483, 5.5484715, 3.1...","[6.3922806, 6.12641, 6.240055, 2.9513688, 2.46...","[[-0.7033329, 1.3870198, -0.78114915], [-0.603...","[[[8.767768, 0.1923849, 0.112969555, 0.2140008...","[-0.13837181, -0.3316361, 0.20996912, 0.108538...","[-2.6398904, 2.3426292, -0.10381953]",1


In [16]:
ds = datasets.load_dataset("hatexplain")

  0%|          | 0/3 [00:00<?, ?it/s]

In [41]:
np.logical_and.reduce(ds["validation"][50]["rationales"], axis=0) * 1

array([0, 1, 0, 0, 0, 0])

In [40]:
for a in ds["validation"][50]["rationales"]:
    print(a)

[0, 1, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 0]


In [43]:
ds["validation"][50]

{'id': '1080898632050724864_twitter',
 'annotators': {'label': [2, 1, 2],
  'annotator_id': [206, 199, 203],
  'target': [['None'], ['None'], ['None']]},
 'rationales': [[0, 1, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0]],
 'post_tokens': ['gt', 'raped', 'by', 'light', '<user>', '<user>']}

In [97]:
def offset_mapping(x):
    o = []
    last_i = 0
    for token in x:
        o.append([last_i, last_i+len(token)])
        last_i = last_i + len(token) + 1
    return o

c = 0
for i, row in df.iterrows():
    imp = row["importance_last_layer_classifier"][:, 0]
    text = " ".join(ds["validation"][i]["post_tokens"])
    t = tokenizer(text, padding=False, max_length=128, truncation=True, return_offsets_mapping=True)
    o_true = offset_mapping(ds["validation"][50]["post_tokens"])
    
    rationales = np.logical_and.reduce(ds["validation"][50]["rationales"], axis=0) * 1
    for j, o in enumerate(t['offset_mapping']):
        if j in imp.argsort()[-5:][::-1]:
            for k, o2 in enumerate(o_true):
                if o[0] >= o2[0] and o[1] <= o2[1] and rationales[k] == 1:
                    print(o)
                    print(o2)
                    print(text[o[0]: o[1]])
                    print(text)
                    c += 1
c, len(df)

(5, 7)
[3, 8]
fu
y si fuera top <number> me faltarían how to get away with murder gossip girl the last ship orphan black downton abbey
(5, 6)
[3, 8]
>
<user> <user> <user> i am a lesbian not queer and quite frankly his disco tits video was repulsive if he thinks being an overweight gay bloke brings out the woman in him then i am offended it pathetic attention seeking
(5, 6)
[3, 8]
>
<user> <user> <user> i love you
(6, 8)
[3, 8]
ni
these niggas ruined the <number> toyota supra image super bowl
(5, 6)
[3, 8]
>
<user> no bra is more trailer trash than that 🤔
(5, 7)
[3, 8]
hs
niggahs ain ’ t solid and these bitches clowned out
(3, 6)
[3, 8]
sha
mudshark and black diversity
(6, 8)
[3, 8]
rk
mudshark and black diversity
(5, 6)
[3, 8]
>
<user> also clapham is full of white graduates who work in the city
(5, 6)
[3, 8]
>
<user> i remember when it was great country cooking and a hillbilly country store now its just another commercial shops
(5, 6)
[3, 8]
>
<user> <user> no that a different talent

(3, 7)
[3, 8]
what
yo what are my muslim followers going through rn your tweets got me feeling hurt too lmao
(5, 6)
[3, 8]
>
<user> but also get ready for my dyke fits sweethart 🥰
(4, 7)
[3, 8]
eak
unspeakable crimes to christians women and girls especially
(5, 6)
[3, 8]
>
<user> that exactly what his nigger ass did 😂
(3, 5)
[3, 8]
do
so do all women if the media teaches them to be whores
(4, 7)
[3, 8]
you
did you know muzzie dont pay taxes or interest
(3, 5)
[3, 8]
co
of couse the jewish qb gets traded to miami
(4, 8)
[3, 8]
jews
the jews want to use the muslims as an army to wipeout us and then they will wipe out most of the muslims
(5, 6)
[3, 8]
>
<user> uk is an islamic state are brits going to take their country back
(4, 6)
[3, 8]
go
lol go back to sucking muslim refugee cock retarded whore
(7, 8)
[3, 8]
>
<number> minutes playing the moslem appeasement ticket wankers
(3, 5)
[3, 8]
ky
if kyrie irving nigger ass thinks he getting a warm welcome in boston he can forget it
(4, 6)
[3,

(5, 6)
[3, 8]
>
<user> <user> just like close friendship is basicly gay relationship
(7, 8)
[3, 8]
<
<user> <user> just like close friendship is basicly gay relationship
(4, 6)
[3, 8]
go
omg go fuck yourself wetback
(4, 5)
[3, 8]
<
for <number> ima stop being a bitch
(5, 6)
[3, 8]
>
<user> fuck you too
(5, 6)
[3, 8]
>
<user> so brave standing against the imaginary nazi
(5, 6)
[3, 8]
>
<user> lmao actually i take that back it depends on how much i like the person 🥴 but even then sucking dick is ghetto
(4, 6)
[3, 8]
do
why do white people say dawg
(3, 5)
[3, 8]
no
so no violence in the northern parts of nigeria
(7, 8)
[3, 8]
i
<user> i paid for them when i got raped in florida
(3, 7)
[3, 8]
pods
airpods are cool and all until you lose those hoes every <number> min
(4, 6)
[3, 8]
we
lol we are triggered because we were banned from everywhere else because we hurt libs feelings too frequently and we are told that hurt feelz is worse than murder nice try kike
(5, 7)
[3, 8]
do
just do not say 

(293, 1922)

In [79]:
offset_mapping(ds["validation"][50]["post_tokens"])

[[0, 2], [3, 8], [9, 11], [12, 17], [18, 24], [25, 31]]

In [51]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True, max_length=128)

{'input_ids': [101, 14181, 15504, 2011, 2422, 1026, 5310, 1028, 1026, 5310, 1028, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 2), (3, 8), (9, 11), (12, 17), (18, 19), (19, 23), (23, 24), (25, 26), (26, 30), (30, 31), (0, 0)]}

In [66]:
t = tokenizer(" ".join(ds["validation"][50]["post_tokens"]), padding=False, max_length=128, truncation=True, return_offsets_mapping=True)
# t = tokenizer(ds["validation"][50]["post_tokens"], padding=False, max_length=128, truncation=True, return_offsets_mapping=True, is_pretokenized=True)

In [68]:
for o in t['offset_mapping']:
    print(o)
    print(" ".join(ds["validation"][50]["post_tokens"])[o[0]:o[1]])

(0, 0)

(0, 2)
gt
(3, 8)
raped
(9, 11)
by
(12, 17)
light
(18, 19)
<
(19, 23)
user
(23, 24)
>
(25, 26)
<
(26, 30)
user
(30, 31)
>
(0, 0)



In [70]:
t['offset_mapping']

[(0, 0),
 (0, 2),
 (3, 8),
 (9, 11),
 (12, 17),
 (18, 19),
 (19, 23),
 (23, 24),
 (25, 26),
 (26, 30),
 (30, 31),
 (0, 0)]